In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('creditCardFraud_28011964_120214.csv')

In [52]:
df.head()
#df.describe()
#df.info()
# this shows all data types are integers, so no need of datatype changing for any column

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
1,50000,1,1,2,37,0,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
2,500000,1,1,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
3,100000,2,2,2,23,0,-1,-1,0,0,...,221,-159,567,380,601,0,581,1687,1542,0
4,140000,2,3,1,28,0,0,2,0,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0


In [ ]:
df.isnull().sum()
# No missing value in the dataset

In [ ]:
#!pip install ydata-profiling
#!pip install pydantic-settings
#!conda install -c conda-forge pandas-profiling=2.6.0

In [5]:
from ydata_profiling import ProfileReport
#from pydantic_settings import BaseSettings

In [ ]:
profile = ProfileReport(df,title='Data Profile Report')
profile

In [ ]:
profile.to_widgets()

In [4]:
X = df.drop(['default payment next month'],axis=1)
y = df['default payment next month']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   LIMIT_BAL  1001 non-null   int64
 1   SEX        1001 non-null   int64
 2   EDUCATION  1001 non-null   int64
 3   MARRIAGE   1001 non-null   int64
 4   AGE        1001 non-null   int64
 5   PAY_0      1001 non-null   int64
 6   PAY_2      1001 non-null   int64
 7   PAY_3      1001 non-null   int64
 8   PAY_4      1001 non-null   int64
 9   PAY_5      1001 non-null   int64
 10  PAY_6      1001 non-null   int64
 11  BILL_AMT1  1001 non-null   int64
 12  BILL_AMT2  1001 non-null   int64
 13  BILL_AMT3  1001 non-null   int64
 14  BILL_AMT4  1001 non-null   int64
 15  BILL_AMT5  1001 non-null   int64
 16  BILL_AMT6  1001 non-null   int64
 17  PAY_AMT1   1001 non-null   int64
 18  PAY_AMT2   1001 non-null   int64
 19  PAY_AMT3   1001 non-null   int64
 20  PAY_AMT4   1001 non-null   int64
 21  PAY_AMT5   100

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=33)

In [8]:
from sklearn.preprocessing import StandardScaler
import pickle
import os
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
#os.makedirs("models/StandardScaler.pkl",exist_ok=True)
file = open("C:\\Users\\Megha Singh\\Documents\\Python\\CreditCardFaultDetection\\models\\StandardScaler.pkl",'wb')
pickle.dump(scaler,file)
file.close()


In [9]:
from sklearn.naive_bayes import GaussianNB 
gb = GaussianNB()

In [10]:
gb.fit(x_train_scaled,y_train)
y_predict = gb.predict(x_test_scaled)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.7940199335548173

In [ ]:
# Hyperparameter Tuning
param_grid={"var_smoothing":[0.1, 0.001, 0.5,0.05,0.01,1e-8,1e-7,1e-6,1e-10,1e-11]}
from sklearn.model_selection import GridSearchCV
gs=GridSearchCV(gb,param_grid=param_grid,cv=5,verbose=3)
gs.fit(x_train_scaled,y_train)

In [13]:
gs.best_params_
gs.best_score_

0.76

In [14]:
gb = GaussianNB(var_smoothing= 0.5)
gb.fit(x_train_scaled,y_train)
y_pred = gb.predict(x_test_scaled)

In [16]:
import pickle
#os.makedirs("models/StandardScaler.pkl",exist_ok=True)
file = open("C:\\Users\\Megha Singh\\Documents\\Python\\CreditCardFaultDetection\\models\\nb_model.pkl",'wb')
pickle.dump(gs,file)
file.close()

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_scaled,y_train)
y_pred = rf.predict(x_test_scaled)
accuracy_score(y_test,y_pred)

0.8205980066445183

In [16]:
param_grid_rf = { "n_estimators": [50,100,130,200],
                    "max_depth": range(3,11,1),
                    "random_state":[0,50,100],
                     "criterion":['gini',"entropy"]
                     }


In [17]:
gs1=GridSearchCV(rf,param_grid=param_grid_rf,cv=5,verbose=3)

In [18]:
gs1.fit(x_train_scaled,y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=0;, score=0.800 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=0;, score=0.771 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=0;, score=0.786 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=0;, score=0.786 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=0;, score=0.779 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=50;, score=0.800 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=50;, score=0.793 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=50;, score=0.786 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=50, random_state=50;, s

In [ ]:
gs1.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'n_estimators': 100,
 'random_state': 0}

In [ ]:
gs1.best_score_

0.8

In [ ]:
from xgboost import XGBClassifier
xf = XGBClassifier()
xf.fit(x_train_scaled,y_train)
y_pred = xf.predict(x_test_scaled)
accuracy_score(y_test,y_pred)

0.7973421926910299

In [ ]:
from scipy.stats import uniform, randint
param = {'max_depth': [2,7,4,5,4], 'eta': [0.03,0.1,0.5,0.8,1], 'objective': ['binary:logistic','reg:linear','multi:softprob'],"n_estimators": [randint(100, 150),randint(100, 150),randint(100, 150)]}
# param['nthread'] = 4
# param['eval_metric'] = 'auc'

In [ ]:
gs2=GridSearchCV(xf,param_grid=param,cv=5,verbose=3)
gs2.fit(x_train_scaled,y_train)

Fitting 5 folds for each of 225 candidates, totalling 1125 fits
[CV 1/5] END eta=0.03, max_depth=2, n_estimators=<scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002968002EC40>, objective=binary:logistic;, score=nan total time=   0.0s
[CV 2/5] END eta=0.03, max_depth=2, n_estimators=<scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002968002EC40>, objective=binary:logistic;, score=nan total time=   0.0s
[CV 3/5] END eta=0.03, max_depth=2, n_estimators=<scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002968002EC40>, objective=binary:logistic;, score=nan total time=   0.0s
[CV 4/5] END eta=0.03, max_depth=2, n_estimators=<scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002968002EC40>, objective=binary:logistic;, score=nan total time=   0.0s
[CV 5/5] END eta=0.03, max_depth=2, n_estimators=<scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002968002EC40>, objective=binary:logistic;, score=nan 

ValueError: 
All the 1125 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1125 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
  File "C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\xgboost\sklearn.py", line 1519, in fit
    self._Booster = train(
  File "C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
  File "C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\xgboost\training.py", l

In [ ]:
gs1.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
gs1.best_params_